In [1]:
import os

In [3]:
# Create proposals json file from ./proposals/*.md files

proposals = []

from pathlib import Path
import re, json

categories = ["unknown", "infrastructure", "integrations", "feature"]

for p in Path('.').glob('proposals/*.md'):
    filename = p.name.replace(".md", "")
    id = filename.replace("wip-", "")
    category = id[:1] # First number in id
    text = p.read_text()
    imagePath = re.search("[\\\/\w]+\.[a-zA-Z]{3,4}", text).group().replace("/", "\\/")
    # WIP-1001 ETH 2.0 support
    title = re.search("\n\#\s+WIP-\d+\s+(.+)\n*", text).group(1)
    body = re.search("\n\#\s+WIP-\d+\s+.+\n*(.*)", text).group(1)

    proposals.append({
        "category": categories[int(category)],
        "image": "https:\/\/raw.githubusercontent.com\/sonsofcrypto\/web3wallet-improvement-proposals\/master"+imagePath,
        "id": id,
        "title": title,
        "body": body,
        "votes": 0,
        "creation_date": "2022-08-28T00:00:00.000Z"
    })
file = open('proposal-list-new.json', 'w')
file.write(json.dumps(proposals))
file.close()

In [4]:
# Create static page from template each proposal ./static/${PROPOSAL_ID}.html
templateFile = open('./static/template.html', 'r')
template = templateFile.read()
templateFile.close()

for p in proposals:
    t = template
    t = t.replace("${ID}", p["id"])
    t = t.replace("${TITLE}", p["title"])
    t = t.replace("${DESCRIPTION}", "web3Wallet improvement proposal " + p["id"])
    t = t.replace("${IMAGE_URL}", p["image"].replace("\\", ""))
    t = t.replace("${CONTENT}", p["body"])
    t = t.replace("${URL}", "https://sonsofcrypto.com/proposals/"+p["id"]+".html")
    file = open('./static/'+p["id"]+'.html', 'w')
    file.write(t)
    file.close()
    print("done: "+'./static/'+p["id"]+'.html')

done: ./static/2022.html
done: ./static/2016.html
done: ./static/2006.html
done: ./static/2012.html
done: ./static/2026.html
done: ./static/2002.html
done: ./static/2013.html
done: ./static/2027.html
done: ./static/2003.html
done: ./static/2023.html
done: ./static/2017.html
done: ./static/2007.html
done: ./static/2028.html
done: ./static/1006.html
done: ./static/3002.html
done: ./static/2018.html
done: ./static/1002.html
done: ./static/3006.html
done: ./static/2008.html
done: ./static/2019.html
done: ./static/1003.html
done: ./static/2009.html
done: ./static/3003.html
done: ./static/3004.html
done: ./static/1004.html
done: ./static/1005.html
done: ./static/3001.html
done: ./static/1001.html
done: ./static/3005.html
done: ./static/2010.html
done: ./static/2024.html
done: ./static/2020.html
done: ./static/2014.html
done: ./static/2004.html
done: ./static/2021.html
done: ./static/2015.html
done: ./static/2005.html
done: ./static/2011.html
done: ./static/2025.html
done: ./static/2001.html


In [107]:
# Download votes (number of tweets with hashtag for given proposal)